# **Cell 1: Install Dependencies**

In [1]:
!pip install -q torch transformers peft trl bitsandbytes accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# **Cell 2: The Training Script**

In [3]:
import os
import json
import torch
import pandas as pd
import logging
from datasets import load_dataset

from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

# Set environment variable to potentially help with memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# --- Constants ---
BASE_MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
NEW_ADAPTER_NAME = "aida-phi3-mini-sre-adapter-v1"
DATASET_FILE = "aida_training_dataset.jsonl"

# --- Load Dataset ---
print(f"Loading dataset from {DATASET_FILE}")
dataset = load_dataset("json", data_files=DATASET_FILE, split="train")
print(f"Successfully loaded {len(dataset)} training samples.")

# Assuming the text data is in a column named 'text', if not, rename it
# You might need to inspect your dataset file to confirm the actual column name
# If your text column has a different name, replace 'text' below with the correct name
if "text" not in dataset.column_names:
    # This is a placeholder, you need to replace 'your_text_column_name'
    # with the actual name of the column containing the text in your JSONL file
    your_text_column_name = list(dataset.column_names)[0] # Assuming the first column is text if 'text' is not found
    print(f"Renaming column '{your_text_column_name}' to 'text'")
    dataset = dataset.rename_column(your_text_column_name, "text")


# --- Configure Training ---
print("--- Initializing Fine-Tuning Pipeline ---")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, # Changed from torch.bfloat16
    bnb_4bit_use_double_quant=False,
)

print(f"Loading base model: {BASE_MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=quant_config,
    trust_remote_code=True,
    device_map={"": 0},
    config={"pretraining_tp": 1} # Set pretraining_tp to 1
)

# Add this line to set _from_remote_code to True
model.config._from_remote_code = True

model.config.use_cache = False
# Removed the line model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["qkv_proj", "o_proj", "gate_up_proj", "down_proj"], # Added target_modules
)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1, # Already set to 1, consider other options if OOM persists
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    logging_steps=1, # Log every step since our dataset is small
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    # Removed dataset_text_field argument
    # Removed max_seq_length argument
    # Removed tokenizer argument
    args=training_args,
    # Removed packing=False,
)

# --- Start Training ---
print("--- Starting model training ---")
trainer.train()
print("--- Model training complete ---")

# --- Save Model ---
print(f"Saving trained adapter to ./{NEW_ADAPTER_NAME}")
trainer.model.save_pretrained(NEW_ADAPTER_NAME)
print("--- Pipeline Finished Successfully ---")

Loading dataset from aida_training_dataset.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

Successfully loaded 1 training samples.
--- Initializing Fine-Tuning Pipeline ---
Loading base model: microsoft/Phi-3-mini-4k-instruct


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yemregultepe (yunusemregultepe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,1.477800


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


--- Model training complete ---
Saving trained adapter to ./aida-phi3-mini-sre-adapter-v1
--- Pipeline Finished Successfully ---
